In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
import requests
import time
import pandas as pd
import datetime

### Сбор датасета новостей через API Alphavantage

In [56]:

def nlp_data(date, api_key='SU9QFO6IITPCANJW'):
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=CRYPTO:BTC&apikey={api_key}&time_from={date}&limit=1000&sort=EARLIEST'
    r = requests.get(url)

    if r.status_code != 200:
        print(f'Error fetching data: {r.status_code}')
        return pd.DataFrame()  

    data = r.json()

    if 'feed' not in data:
        print('No "feed" key in the response.')
        return pd.DataFrame()  # Возвращаем пустой DataFrame в случае отсутствия ключа

    if not data['feed']:
        print('Feed is empty.')
        return pd.DataFrame()  # Возвращаем пустой DataFrame в случае пустого списка

    df = pd.DataFrame(data['feed'])

     # Преобразуем формат времени в стандартный формат даты
    df['date_published'] = pd.to_datetime(df['time_published'], format='%Y%m%dT%H%M%S')
    
    # Удалим старый столбец времени
    df.drop('time_published', axis=1, inplace=True)
    
    # Сортируем значения по дате
    df_sorted = df.sort_values(by='date_published')

    # Найти последнюю дату и время в полученных данных
    last_entry = df_sorted.iloc[-1]
    last_time = last_entry['date_published']

    # Преобразуем последнюю дату и время в нужный формат
    last_time_str = last_time.strftime('%Y%m%dT%H%M')

    # Приостановка перед следующим запросом
    time.sleep(3)

    # Рекурсивный вызов для получения следующих данных
    df_2 = nlp_data(last_time_str, api_key=api_key)

    # Объединяем текущие данные с полученными рекурсивно
    result = pd.concat([df_sorted, df_2], axis=0)

    return result

# Запуск функции
a = nlp_data('20210410T0130', api_key='SU9QFO6IITPCANJW')

print(a)


No "feed" key in the response.
                                                 title  \
0         Geopolitical Risk Returns for Global Markets   
1    Asian shares slip, oil surges again as Russia ...   
2    Business Highlights: Lobbyists leaving, rate h...   
3    Millions for Crypto Start-Ups, No Real Names N...   
4    How to Make  ( and Keep )  Money in the Market...   
..                                                 ...   
716  Chinese financial institutions' NFT guidelines...   
717        UPI Approves Another 60M Users For WhatsApp   
718  Bitcoin retains $41K as BTC exchange withdrawa...   
719  Lucid Or Elusive Dreams: Generation Z To Alpha...   
720  Why Russian Oligarchs Can't Evade Sanctions Th...   

                                                   url  \
0    https://www.nytimes.com/2022/03/01/business/de...   
1    https://www.moneycontrol.com/news/business/mar...   
2    https://apnews.com/article/russia-ukraine-tech...   
3    https://www.nytimes.com/2022/03/02/

In [151]:
nlp_data = pd.concat([a1,a2, a3, a4], axis=0) # a1, a2, a3, a4 - подгружались с других машин из-за ограничения запросов в день

In [217]:
nlp_data

,title,url,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,date_published
0,Geopolitical Risk Returns for Global Markets,https://www.nytimes.com/2022/03/01/business/de...,[],Investors are weighing the implications of Rus...,https://static01.nyt.com/images/2022/03/01/mul...,New York Times,Dealbook,www.nytimes.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.138797,Neutral,"[{'ticker': 'NFLX', 'relevance_score': '0.0189...",2022-03-01
1,"Asian shares slip, oil surges again as Russia ...",https://www.moneycontrol.com/news/business/mar...,[],"As global sanctions against Moscow tighten, th...",https://images.moneycontrol.com/static-mcnews/...,Money Control,Markets,www.moneycontrol.com,"[{'topic': 'Energy & Transportation', 'relevan...",-0.031243,Neutral,"[{'ticker': 'EQNR', 'relevance_score': '0.0639...",2022-03-02
2,"Business Highlights: Lobbyists leaving, rate h...",https://apnews.com/article/russia-ukraine-tech...,[],"Business Highlights: Lobbyists leaving, rate h...",https://storage.googleapis.com/afs-prod/media/...,Associated Press,Business,apnews.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.174572,Somewhat-Bearish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",2022-03-02
3,"Millions for Crypto Start-Ups, No Real Names N...",https://www.nytimes.com/2022/03/02/technology/...,[],Investors give money to pseudonymous developer...,https://static01.nyt.com/images/2022/02/23/bus...,New York Times,Dealbook,www.nytimes.com,"[{'topic': 'Financial Markets', 'relevance_sco...",-0.036470,Neutral,"[{'ticker': 'NYT', 'relevance_score': '0.0234'...",2022-03-02
4,How to Make ( and Keep ) Money in the Market...,https://moneymorning.com/2022/03/02/how-to-mak...,"[chris_johnson, Chris Johnson]",Chris Johnson shows you what to look for in or...,https://moneymorning.com/wp-content/blogs.dir/...,Money Morning,n/a,moneymorning.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.034280,Neutral,"[{'ticker': 'X', 'relevance_score': '0.112506'...",2022-03-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,Bitcoin Runes Resurgence? Why Asia's Instituti...,https://decrypt.co/233430/bitcoin-runes-resurg...,"['NFT Dudu', 'Sander Lutz']",The Bitcoin meme coin protocol cooled off quic...,https://cdn.decrypt.co/resize/1024/height/512/...,Decrypt.co,NaN,decrypt.co,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.269231,Somewhat-Bullish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",2024-06-02 20:07:28
8124,Bitcoin Runes Resurgence? Why Asia's Instituti...,https://decrypt.co/233430/bitcoin-runes-resurg...,"['NFT Dudu', 'Sander Lutz']",The Bitcoin meme coin protocol cooled off quic...,https://cdn.decrypt.co/resize/1024/height/512/...,Decrypt.co,NaN,decrypt.co,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.269231,Somewhat-Bullish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",2024-06-02 20:07:28
8125,Bitcoin Runes Resurgence? Why Asia's Instituti...,https://decrypt.co/233430/bitcoin-runes-resurg...,"['NFT Dudu', 'Sander Lutz']",The Bitcoin meme coin protocol cooled off quic...,https://cdn.decrypt.co/resize/1024/height/512/...,Decrypt.co,NaN,decrypt.co,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.269231,Somewhat-Bullish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",2024-06-02 20:07:28
8126,Bitcoin Runes Resurgence? Why Asia's Instituti...,https://decrypt.co/233430/bitcoin-runes-resurg...,"['NFT Dudu', 'Sander Lutz']",The Bitcoin meme coin protocol cooled off quic...,https://cdn.decrypt.co/resize/1024/height/512/...,Decrypt.co,NaN,decrypt.co,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.269231,Somewhat-Bullish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",2024-06-02 20:07:28


In [153]:
data_nlp = nlp_data.reset_index(drop=True)

In [154]:
data_nlp


,title,url,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,date_published
0,Geopolitical Risk Returns for Global Markets,https://www.nytimes.com/2022/03/01/business/de...,[],Investors are weighing the implications of Rus...,https://static01.nyt.com/images/2022/03/01/mul...,New York Times,Dealbook,www.nytimes.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.138797,Neutral,"[{'ticker': 'NFLX', 'relevance_score': '0.0189...",2022-03-01
1,"Asian shares slip, oil surges again as Russia ...",https://www.moneycontrol.com/news/business/mar...,[],"As global sanctions against Moscow tighten, th...",https://images.moneycontrol.com/static-mcnews/...,Money Control,Markets,www.moneycontrol.com,"[{'topic': 'Energy & Transportation', 'relevan...",-0.031243,Neutral,"[{'ticker': 'EQNR', 'relevance_score': '0.0639...",2022-03-02
2,"Business Highlights: Lobbyists leaving, rate h...",https://apnews.com/article/russia-ukraine-tech...,[],"Business Highlights: Lobbyists leaving, rate h...",https://storage.googleapis.com/afs-prod/media/...,Associated Press,Business,apnews.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.174572,Somewhat-Bearish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",2022-03-02
3,"Millions for Crypto Start-Ups, No Real Names N...",https://www.nytimes.com/2022/03/02/technology/...,[],Investors give money to pseudonymous developer...,https://static01.nyt.com/images/2022/02/23/bus...,New York Times,Dealbook,www.nytimes.com,"[{'topic': 'Financial Markets', 'relevance_sco...",-0.036470,Neutral,"[{'ticker': 'NYT', 'relevance_score': '0.0234'...",2022-03-02
4,How to Make ( and Keep ) Money in the Market...,https://moneymorning.com/2022/03/02/how-to-mak...,"[chris_johnson, Chris Johnson]",Chris Johnson shows you what to look for in or...,https://moneymorning.com/wp-content/blogs.dir/...,Money Morning,n/a,moneymorning.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.034280,Neutral,"[{'ticker': 'X', 'relevance_score': '0.112506'...",2022-03-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47895,Bitcoin Runes Resurgence? Why Asia's Instituti...,https://decrypt.co/233430/bitcoin-runes-resurg...,"['NFT Dudu', 'Sander Lutz']",The Bitcoin meme coin protocol cooled off quic...,https://cdn.decrypt.co/resize/1024/height/512/...,Decrypt.co,NaN,decrypt.co,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.269231,Somewhat-Bullish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",2024-06-02 20:07:28
47896,Bitcoin Runes Resurgence? Why Asia's Instituti...,https://decrypt.co/233430/bitcoin-runes-resurg...,"['NFT Dudu', 'Sander Lutz']",The Bitcoin meme coin protocol cooled off quic...,https://cdn.decrypt.co/resize/1024/height/512/...,Decrypt.co,NaN,decrypt.co,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.269231,Somewhat-Bullish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",2024-06-02 20:07:28
47897,Bitcoin Runes Resurgence? Why Asia's Instituti...,https://decrypt.co/233430/bitcoin-runes-resurg...,"['NFT Dudu', 'Sander Lutz']",The Bitcoin meme coin protocol cooled off quic...,https://cdn.decrypt.co/resize/1024/height/512/...,Decrypt.co,NaN,decrypt.co,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.269231,Somewhat-Bullish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",2024-06-02 20:07:28
47898,Bitcoin Runes Resurgence? Why Asia's Instituti...,https://decrypt.co/233430/bitcoin-runes-resurg...,"['NFT Dudu', 'Sander Lutz']",The Bitcoin meme coin protocol cooled off quic...,https://cdn.decrypt.co/resize/1024/height/512/...,Decrypt.co,NaN,decrypt.co,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.269231,Somewhat-Bullish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",2024-06-02 20:07:28


In [219]:
api_nlp = data_nlp

In [220]:
api_nlp.to_csv('data_api_nlp.csv', index=False)

In [178]:
len(data_nlp[['summary', 'title']].index[data_nlp[['summary']].duplicated()]) # подсчет дубликатов

1673

In [183]:
data_for_nlp = data_nlp[[ 'title', 'summary', 'source' , 'date_published' ]] # отсеяли ненужные столбцы

In [186]:
data_for_nlp = data_for_nlp.drop_duplicates(keep='last') #удаление дубликатолв

In [188]:
data_for_nlp['date_published'] = pd.to_datetime(data_for_nlp['date_published'])

# Удаление времени из столбца дат, чтобы оставить только даты
data_for_nlp['date_published'] = data_for_nlp['date_published'].dt.date

C:\Users\Shox\AppData\Local\Temp\ipykernel_41228\740761232.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_nlp['date_published'] = pd.to_datetime(data_for_nlp['date_published'])
C:\Users\Shox\AppData\Local\Temp\ipykernel_41228\740761232.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_nlp['date_published'] = data_for_nlp['date_published'].dt.date


#### Колонки в данных ниже:
- title - заголовк новости,
- summary - краткое содержание новости,
- source - источник новости,
- date_published - дата публикации

In [189]:
data_for_nlp

,title,summary,source,date_published
0,Geopolitical Risk Returns for Global Markets,Investors are weighing the implications of Rus...,New York Times,2022-03-01
1,"Asian shares slip, oil surges again as Russia ...","As global sanctions against Moscow tighten, th...",Money Control,2022-03-02
2,"Business Highlights: Lobbyists leaving, rate h...","Business Highlights: Lobbyists leaving, rate h...",Associated Press,2022-03-02
3,"Millions for Crypto Start-Ups, No Real Names N...",Investors give money to pseudonymous developer...,New York Times,2022-03-02
4,How to Make ( and Keep ) Money in the Market...,Chris Johnson shows you what to look for in or...,Money Morning,2022-03-02
...,...,...,...,...
47881,"BNB, AR, XMR, and TIA set to rally - Will Bitc...",Bitcoin price action could remain sideways for...,Cointelegraph,2024-06-02
47882,Scaramucci Warns Biden's Veto Of Crypto Custod...,Former White House Director of Communications ...,Benzinga,2024-06-02
47883,How Crypto SuperPACs Will Change Crypto Policy,As the crypto industry continues to rapidly gr...,Forbes,2024-06-02
47884,Massive Shiba Inu Burn Ignites Hope For Price ...,Dog-themed cryptocurrency Shiba Inu SHIB/USD h...,Benzinga,2024-06-02


Объединим данные группировкой по дням, т.е. новости в течение одного дня объединятся в одно наблюдение соответствующего дня,и проделаем это только  с   колонкой с содержанием новостей, так как она самая содержательная.

In [196]:
data_brt = data_for_nlp.groupby('date_published')['summary'].apply(' '.join).reset_index()

print(result)

   date_published                                            summary
0      2022-03-01  Investors are weighing the implications of Rus...
1      2022-03-02  As global sanctions against Moscow tighten, th...
2      2022-03-03  Yellen suggested the cryptocurrency sector was...
3      2022-03-04  Live updates: Dow tumbles after Russian nuclea...
4      2022-03-05  Let's take a look at some ETF areas that inves...
5      2022-03-06  David Zeiler tells you everything you need to ...
6      2022-03-07  The sustainability of cryptocurrencies as a re...
7      2022-03-08  Major coins traded in the red on Monday evenin...
8      2022-03-09  Could crypto pioneer inclusive organizations f...
9      2022-03-10  President Biden's executive order on cryptocur...
10     2022-03-11  The idea is to help bring all the elements of ...
11     2022-03-12  The co-founder says the number of people getti...
12     2022-03-13  Decentralized finance provides the necessary f...
13     2022-03-14  It may have mor

In [197]:
data_brt

,date_published,summary
0,2022-03-01,Investors are weighing the implications of Rus...
1,2022-03-02,"As global sanctions against Moscow tighten, th..."
2,2022-03-03,Yellen suggested the cryptocurrency sector was...
3,2022-03-04,Live updates: Dow tumbles after Russian nuclea...
4,2022-03-05,Let's take a look at some ETF areas that inves...
...,...,...
820,2024-05-29,"AI crypto tokens are ""quiet now,"" bleeding red..."
821,2024-05-30,A judge has sanctioned the SEC for its failed ...
822,2024-05-31,Bitcoin markets are safe from a Mt Gox dump fo...
823,2024-06-01,Franklin Templeton has started the spot Ethere...


In [198]:
data_brt.set_index('date_published', inplace=True)

### Применение модели  bart-large-mnli  к датасету

#### Эта модель - предварительно обученный классификатор BART-Large MNLI, использует encoder предварительно обученной модели BART-Large и дообучена на наборе данных MultiNLI, который содержит 433 тысячи пар предложений, аннотированных текстовой информацией. На вход модели подается текст и гипотеза/гипотезы и для каждой она выдает число (вероятность) от 0 до 1 насколько текст соответствует гипотезе. В нашем случае взята гипотеза : "This example is bullish for Bitcoin" (аналогичный подход с такой же гипотезой использовался в https://arxiv.org/abs/2311.14759).


In [200]:
import torch
print(torch.cuda.is_available())

True


In [209]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [210]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=0)

In [1]:
candidate_labels = ['This example is bullish for Bitcoin'] # сформировали гипотезу

In [212]:
def classify_text(text):
    result = classifier(text, candidate_labels)
    return result['scores'][0]

In [213]:
data_brt['classification'] = data_brt['summary'].apply(classify_text)# Применение модели для каждого наблюдения

In [214]:
data_brt

,summary,classification
date_published,,
2022-03-01,Investors are weighing the implications of Rus...,0.015050
2022-03-02,"As global sanctions against Moscow tighten, th...",0.132170
2022-03-03,Yellen suggested the cryptocurrency sector was...,0.619985
2022-03-04,Live updates: Dow tumbles after Russian nuclea...,0.371239
2022-03-05,Let's take a look at some ETF areas that inves...,0.212605
...,...,...
2024-05-29,"AI crypto tokens are ""quiet now,"" bleeding red...",0.595708
2024-05-30,A judge has sanctioned the SEC for its failed ...,0.885802
2024-05-31,Bitcoin markets are safe from a Mt Gox dump fo...,0.378112


В столбце classification лежат оценки модели по новостям на каждый из дней

In [221]:
data_brt.to_csv('data_from_nlp.csv', index=True)